In [1]:
from xml.etree import ElementTree
from PIL import Image
import numpy as np
import shutil
import os
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import layers
from keras import models
from collections import Counter
from math import ceil, floor
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
from keras.layers import Dropout, Conv3D, BatchNormalization, MaxPooling3D, AveragePooling2D, Activation
from PIL import Image, ImageOps

Using TensorFlow backend.
/home/jose/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jose/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jose/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jose/anaconda3/lib/python3.7/site-packages/tensorflow/py

In [2]:
cat_dict = {'acetone':0, 'acetonitrile': 1, 'chloroform':2, 'dichloromethane':3,'diethylether':4,'ethanol':5,'ethylacetate':6,'heptane':7,'hexane':8,'methanol':9,'toluene':10}
cat_dict_reverse = dict(map(reversed,cat_dict.items()))

In [3]:
def extract_boxes(filename):
    # load and parse the file
    tree = ElementTree.parse(filename)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    boxes = list()
    for box in root.findall('.//bndbox'):
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        coors = [xmin, ymin, xmax, ymax]
        boxes.append(coors)
    # extract image dimensions
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height

In [4]:
path = '/media/jose/USB-HDD/Tese/Videos_Original_Resolution_Seq32'

bboxes_per_video = {}
first_frame = {}

for cat in cat_dict:
    bboxes_per_video[cat] = {}
    first_frame[cat] = {}
    for video_no in range(1,6):
        bboxes_per_video[cat][video_no] = 0
        first_file = 1000
        for file in os.listdir(path+'/'+cat+'/seq'+str(video_no)):
            if file.endswith('.xml'):
                aux_num = int(file[5:-4])
                if aux_num < first_file:
                    first_file = aux_num
        first_frame[cat][video_no] = first_file
        xml_name = 'frame'+str(first_file)+'.xml'
        #xml_name = cat+str(video_no)+'frame'+str(0)+'.xml'
        b, _, _ = extract_boxes(path+'/'+cat+'/seq'+str(video_no)+'/'+xml_name)
        bboxes_per_video[cat][video_no] = b

In [6]:
shutil.rmtree('sequences_75x75')
aux1 = ['train','val','test']

os.mkdir('sequences_75x75')
for i in aux1:
    os.mkdir('sequences_75x75/'+i)

In [7]:
#generate train sequences
image_sequences = {}
seq_counter = 0

for cat in bboxes_per_video:
    print(cat)
    image_sequences[cat] = {}
    #for video in bboxes_per_video[cat]:
    for video in range(1,4): #only train sequences for now
        aux_im_list = list()
        for i in range(first_frame[cat][video],first_frame[cat][video]+32):
            im_path = path+'/'+cat+'/seq'+str(video)+'/frame'+str(i)+'.png'
            aux_im_list.append(Image.open(im_path))
        image_sequences[cat][video] = list()
        for drop_no in range(len(bboxes_per_video[cat][video])):
            drop_seq = list()
            for frame_no in range(0,32):
                #drop_seq.append(img_to_array(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((39,39)))/255)
                #drop_seq.append(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((39,39)))
                drop_seq.append(img_to_array(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((75,75))))
                #drop_seq.append(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((39,39)))
            #image_sequences[cat][video].append(drop_seq)
            aux = (np.asarray(drop_seq).astype(np.uint8), to_categorical(cat_dict[cat],11))
            np.save('sequences_75x75/train/seq_'+str(seq_counter)+'.npy',aux)
            seq_counter += 1

acetone
acetonitrile
chloroform
dichloromethane
diethylether
ethanol
ethylacetate
heptane
hexane
methanol
toluene


In [8]:
#generate val sequences
image_sequences = {}
seq_counter = 0

for cat in bboxes_per_video:
    print(cat)
    image_sequences[cat] = {}
    #for video in bboxes_per_video[cat]:
    for video in range(4,5): #only val sequences for now
        aux_im_list = list()
        for i in range(first_frame[cat][video],first_frame[cat][video]+32):
            im_path = path+'/'+cat+'/seq'+str(video)+'/frame'+str(i)+'.png'
            aux_im_list.append(Image.open(im_path))
        image_sequences[cat][video] = list()
        for drop_no in range(len(bboxes_per_video[cat][video])):
            drop_seq = list()
            for frame_no in range(0,32):
                #drop_seq.append(img_to_array(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((39,39)))/255)
                #drop_seq.append(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((39,39)))
                drop_seq.append(img_to_array(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((75,75))))
                #drop_seq.append(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((39,39)))
            #image_sequences[cat][video].append(drop_seq)
            aux = (np.asarray(drop_seq).astype(np.uint8), to_categorical(cat_dict[cat],11))
            np.save('sequences_75x75/val/seq_'+str(seq_counter)+'.npy',aux)
            seq_counter += 1

acetone
acetonitrile
chloroform
dichloromethane
diethylether
ethanol
ethylacetate
heptane
hexane
methanol
toluene


In [5]:
#generate test sequences
image_sequences = {}
seq_counter = 0

for cat in bboxes_per_video:
    print(cat)
    image_sequences[cat] = {}
    #for video in bboxes_per_video[cat]:
    for video in range(5,6): #only val sequences for now
        aux_im_list = list()
        for i in range(first_frame[cat][video],first_frame[cat][video]+32):
            im_path = path+'/'+cat+'/seq'+str(video)+'/frame'+str(i)+'.png'
            aux_im_list.append(Image.open(im_path))
        image_sequences[cat][video] = list()
        for drop_no in range(len(bboxes_per_video[cat][video])):
            drop_seq = list()
            for frame_no in range(0,32):
                #drop_seq.append(img_to_array(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((39,39)))/255)
                #drop_seq.append(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((39,39)))
                drop_seq.append(img_to_array(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((75,75))))
                #drop_seq.append(aux_im_list[frame_no].crop(tuple(bboxes_per_video[cat][video][drop_no])).resize((39,39)))
            #image_sequences[cat][video].append(drop_seq)
            aux = (np.asarray(drop_seq).astype(np.uint8), to_categorical(cat_dict[cat],11))
            np.save('sequences_75x75/test/seq_'+str(seq_counter)+'.npy',aux)
            seq_counter += 1

acetone
acetonitrile
chloroform
dichloromethane
diethylether
ethanol
ethylacetate
heptane
hexane
methanol
toluene
